In [1]:
import pandas as pd

# 파일 경로
file_path = '../ExcelFiles/LectureByClass.xlsx'

# 엑셀 파일 읽기
data = pd.read_excel(file_path)

# 데이터의 첫 몇 줄을 출력하여 구조 파악
data.head(), data.columns

(    번호 학년/가진급학년  교과과정      학수강좌번호         교과목명   학점     개설학과(부)/전공 담당교원  \
 0  1.0        1  공통교양  RGC0005-13   기술보고서작성및발표  3.0     공과대학 건축공학부  최현경   
 1  2.0        3    전공  ISE4002-01      생산및운영관리  3.0  공과대학 산업시스템공학과  전성범   
 2  3.0        4    전공  ISE4026-02  산업시스템공학종합설계  3.0  공과대학 산업시스템공학과  서용윤   
 3  4.0      3,4    전공  CEN4077-01   고분자소재 및 응용  3.0   공과대학 화공생물공학과  이상호   
 4  5.0      3,4    전공  CEN4081-02   화공생물공학종합설계  3.0   공과대학 화공생물공학과  홍성규   
 
         호수   좌석수                      요일/시간  Unnamed: 11  인원수  층  
 0  405-127  98.0               목09:00-12:00          NaN   44  1  
 1  405-127  98.0  화13:30-15:00,목13:30-15:00          NaN   85  1  
 2  405-127  98.0               금09:00-12:00          NaN   35  1  
 3  405-127  98.0  월13:30-15:00,목12:00-13:30          NaN   11  1  
 4  405-127  98.0               금12:00-15:00          NaN   48  1  ,
 Index(['번호', '학년/가진급학년', '교과과정', '학수강좌번호', '교과목명', '학점', '개설학과(부)/전공', '담당교원',
        '호수', '좌석수', '요일/시간', 'Unnamed: 11', '인원수', '

In [19]:
import re

# 요일을 숫자로 변환하는 함수
def day_to_number(day):
    days = {'월': 1, '화': 2, '수': 3, '목': 4, '금': 5, '토': 6, '일': 7}
    return days.get(day, 0)

# 시간 정보 분리 및 정리 함수
def parse_time_info(time_str):
    pattern = r'(\w)(\d{2}):(\d{2})-(\d{2}):(\d{2})'
    matches = re.findall(pattern, time_str)
    time_info = []
    for match in matches:
        day, start_hr, start_min, end_hr, end_min = match
        start_time = int(start_hr + start_min)
        duration = (int(end_hr) * 60 + int(end_min)) - (int(start_hr) * 60 + int(start_min))
        time_info.append([day_to_number(day), start_time, duration])
    return time_info

# 강의실별로 데이터 정리 (강의 코드를 '호수' 열의 뒷 3자리를 사용)
def organize_data_with_code(df):
    time_table = {}
    for _, row in df.iterrows():
        # '호수' 열에서 강의 코드 추출 (뒷 3자리)
        classroom_code = int(row['호수'].split('-')[-1])
        time_slots = parse_time_info(row['요일/시간'])
        for slot in time_slots:
            if classroom_code not in time_table:
                time_table[classroom_code] = []
            time_table[classroom_code].append(slot + [row['인원수']])
    return time_table

# 강의 데이터 정리
organized_time_table = organize_data_with_code(data)
organized_time_table[127][:5]  # 예시 출력 (강의실 '127'의 첫 5개 강의 정보 확인)

[[4, 900, 180, 44],
 [2, 1330, 90, 85],
 [4, 1330, 90, 85],
 [5, 900, 180, 35],
 [1, 1330, 90, 11]]

In [20]:
organized_time_table

{127: [[4, 900, 180, 44],
  [2, 1330, 90, 85],
  [4, 1330, 90, 85],
  [5, 900, 180, 35],
  [1, 1330, 90, 11],
  [4, 1200, 90, 11],
  [5, 1200, 180, 48],
  [2, 1030, 90, 28],
  [2, 1600, 170, 76],
  [1, 1800, 150, 77],
  [5, 1800, 150, 77],
  [3, 1200, 180, 20],
  [5, 1500, 180, 17]],
 134: [[1, 1500, 330, 25],
  [3, 1500, 330, 14],
  [4, 1500, 330, 26],
  [5, 1500, 330, 25]],
 138: [[1, 1600, 220, 25],
  [2, 1600, 220, 24],
  [3, 1600, 220, 22],
  [1, 1300, 90, 25],
  [3, 1030, 90, 25],
  [1, 1030, 90, 19],
  [3, 1300, 90, 19],
  [4, 1530, 225, 8],
  [1, 1000, 90, 7],
  [2, 1000, 90, 7]],
 240: [[2, 1500, 330, 14],
  [3, 1500, 330, 24],
  [4, 1500, 330, 23],
  [5, 1500, 330, 25]],
 250: [[5, 900, 180, 44],
  [3, 1300, 90, 41],
  [2, 1530, 175, 29],
  [1, 1530, 90, 21],
  [3, 1530, 90, 21],
  [1, 900, 90, 39],
  [3, 900, 90, 39],
  [1, 1200, 90, 40],
  [1, 1030, 90, 37],
  [2, 1330, 90, 37],
  [1, 1330, 90, 34],
  [3, 1030, 90, 34],
  [5, 1200, 90, 19],
  [2, 1030, 90, 39],
  [4, 1030, 